task name: Implement SgdNonCalibratedWithOptions
task description: Implement the provided C# code for SgdNonCalibratedWithOptions as a dotnet interactive notebook.

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

In [ ]:
var mlContext = new MLContext(seed: 0);
var options = new Microsoft.ML.Trainers.SgdNonCalibratedTrainer.Options
{
    LearningRate = 0.01,
    NumberOfIterations = 10,
    L2Regularization = 1e-7f
};

In [ ]:
var dataPoints = GenerateRandomDataPoints(1000);
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

private static IEnumerable<DataPoint> GenerateRandomDataPoints(int count, int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)random.NextDouble();
    for (int i = 0; i < count; i++)
    {
        var label = randomFloat() > 0.5f;
        yield return new DataPoint
        {
            Label = label,
            Features = Enumerable.Repeat(label, 50)
                .Select(x => x ? randomFloat() : randomFloat() + 0.03f).ToArray()
        };
    }
}

private class DataPoint
{
    public bool Label { get; set; }
    [VectorType(50)]
    public float[] Features { get; set; }
}

In [ ]:
var pipeline = mlContext.BinaryClassification.Trainers.SgdNonCalibrated(options);
var model = pipeline.Fit(trainingData);

In [ ]:
var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(500, seed: 123));
var transformedTestData = model.Transform(testData);

In [ ]:
var predictions = mlContext.Data.CreateEnumerable<Prediction>(transformedTestData, reuseRowObject: false).ToList();
var metrics = mlContext.BinaryClassification.EvaluateNonCalibrated(transformedTestData);
PrintMetrics(metrics);

private class Prediction
{
    public bool Label { get; set; }
    public bool PredictedLabel { get; set; }
}

private static void PrintMetrics(BinaryClassificationMetrics metrics)
{
    Console.WriteLine($"Accuracy: {metrics.Accuracy:F2}");
    Console.WriteLine($"AUC: {metrics.AreaUnderRocCurve:F2}");
    Console.WriteLine($"F1 Score: {metrics.F1Score:F2}");
    Console.WriteLine($"Negative Precision: {metrics.NegativePrecision:F2}");
    Console.WriteLine($"Negative Recall: {metrics.NegativeRecall:F2}");
    Console.WriteLine($"Positive Precision: {metrics.PositivePrecision:F2}");
    Console.WriteLine($"Positive Recall: {metrics.PositiveRecall:F2}\n");
    Console.WriteLine(metrics.ConfusionMatrix.GetFormattedConfusionTable());
}